In [1]:
from config import *
import tensorflow as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
import IMDBLoader
import numpy as np
from tensorflow.losses import Reduction
import chars2vec

Using TensorFlow backend.


In [2]:
train_dir = "/home/hadoop/IMDB/aclImdb/train/"
test_dir = "/home/hadoop/IMDB/aclImdb/test/"
train_set, test_set = IMDBLoader.load_data(train_dir, test_dir)
train_set.extend(test_set[:20000])
valid_set = test_set[20000:21000]
test_set = test_set[21000:]
train_reader = IMDBLoader.IMDBReader(train_set, 20)
valid_reader = IMDBLoader.IMDBReader(valid_set, 20)
test_reader = IMDBLoader.IMDBReader(test_set, 20)

In [3]:
def linear(input_, output_size, scope=None):
    '''
    Linear map: output[k] = sum_i(Matrix[k, i] * args[i] ) + Bias[k]

    Args:
        args: a tensor or a list of 2D, batch x n, Tensors.
    output_size: int, second dimension of W[i].
    scope: VariableScope for the created subgraph; defaults to "Linear".
  Returns:
    A 2D Tensor with shape [batch x output_size] equal to
    sum_i(args[i] * W[i]), where W[i]s are newly created matrices.
  Raises:
    ValueError: if some of the arguments has unspecified or wrong shape.
  '''

    shape = input_.get_shape().as_list()
    if len(shape) != 2:
        raise ValueError("Linear is expecting 2D arguments: %s" % str(shape))
    if not shape[1]:
        raise ValueError("Linear expects shape[1] of arguments: %s" % str(shape))
    input_size = shape[1]

    # Now the computation.
    with tf.variable_scope(scope or "SimpleLinear", reuse=tf.AUTO_REUSE):
        matrix = tf.get_variable("Matrix", [output_size, input_size], dtype=input_.dtype)
        bias_term = tf.get_variable("Bias", [output_size], dtype=input_.dtype)
    return tf.matmul(input_, tf.transpose(matrix)) + bias_term


In [4]:
class SentiModel:
    def __init__(self, max_word_length, kernels, kernel_features,
                            num_highway_layers, embedding_dim):
        self.max_word_length = max_word_length
        self.kernels = kernels
        self.kernel_features = kernel_features
        self.num_highway_layers = num_highway_layers
        self.embedding_dim = embedding_dim
    
    def __call__(self, input_words):
        with tf.variable_scope('Embedding', reuse=tf.AUTO_REUSE):
            # [batch_size x max_word_length, num_unroll_steps, char_embed_size]
            input_cnn = self.tdnn(input_words, self.kernels, self.kernel_features)
            ''' Maybe apply Highway '''
    #             if num_highway_layers > 0:
            assert self.num_highway_layers > 0
            input_cnn = self.highway(input_cnn, input_cnn.get_shape()[-1], num_layers=self.num_highway_layers, scope="CNN_OUT")
            # fcn_layer = tf.layers.Dense(self.embedding_dim, activation=tf.keras.activations.sigmoid, trainable=True)
            # final_embedding = fcn_layer(input_cnn) # [None, max_word_num, hidden_dim]
        print("input_cnn:", input_cnn)
        # print("final_embedding", input_cnn)
        return input_cnn
    
    def conv2d(self, input_, output_dim, k_h, k_w, name="conv2d"):
        with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
            w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim])
            b = tf.get_variable('b', [output_dim])
        return tf.nn.conv2d(input_, w, strides=[1, 1, 1, 1], padding='VALID') + b

    def highway(self, input_, size, num_layers=1, bias=-2.0, f=tf.nn.relu, scope='Highway'):
        """Highway Network (cf. http://arxiv.org/abs/1505.00387).
        t = sigmoid(Wy + b)
        z = t * g(Wy + b) + (1 - t) * y
        where g is nonlinearity, t is transform gate, and (1 - t) is carry gate.
        """
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            for idx in range(num_layers):
                g = f(linear(input_, size, scope='highway_lin_%d' % idx))

                t = tf.sigmoid(linear(input_, size, scope='highway_gate_%d' % idx) + bias)

                output = t * g + (1. - t) * input_
                input_ = output
        return output

    def tdnn(self, input_, kernels, kernel_features, scope='TDNN'):
        '''
        :input:           input float tensor of shape [(batch_size*num_unroll_steps), max_word_length, embed_size]
        :kernels:         array of kernel sizes
        :kernel_features: array of kernel feature sizes (parallel to kernels)
        '''
        assert len(kernels) == len(kernel_features), 'Kernel and Features must have the same size'
        max_word_length = input_.get_shape()[1]
        embed_size = input_.get_shape()[-1]
        # input_: [batch_size*num_unroll_steps, 1, max_word_length, embed_size]
        input_ = tf.expand_dims(input_, 1)
        layers = []
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            for kernel_size, kernel_feature_size in zip(kernels, kernel_features):
                reduced_length = max_word_length - kernel_size + 1
                # [batch_size*num_unroll_steps, 1, reduced_length, kernel_feature_size]
                conv = self.conv2d(input_, kernel_feature_size, 1, kernel_size, name="kernel_%d" % kernel_size)
                # [batch_size*num_unroll_steps, 1, 1, kernel_feature_size]
                pool = tf.nn.max_pool(tf.tanh(conv), [1, 1, reduced_length, 1], [1, 1, 1, 1], 'VALID')
                layers.append(tf.squeeze(pool, [1, 2]))
            if len(kernels) > 1:
                output = tf.concat(layers, 1)
            else:
                output = layers[0]
        return output

In [5]:
class SentimentModel:
    def __init__(self, hidden_dim):
        self.hidden_dim = hidden_dim
        self.drop_out_prob_keep = tf.placeholder(tf.float32, name="lstm_char_net_dp")
        self.fw_cell = self.create_rnn_cell(self.hidden_dim)
        self.bw_cell = self.create_rnn_cell(self.hidden_dim)
    
    def __call__(self, sentence, x_len, fw_init, bw_init): 
    #sentence: [batchsize, max_word_num, embedding_dim]
    #x_len: [batch_size]
        with tf.variable_scope('Embedding', reuse=tf.AUTO_REUSE):
            (fw_outs, bw_outs), (fw_final, bw_final) = tf.nn.bidirectional_dynamic_rnn(
                                        self.fw_cell,
                                        self.bw_cell,
                                        sentence,
                                        sequence_length=x_len,
                                        initial_state_fw=fw_init,
                                        initial_state_bw=bw_init,
                                        dtype=None,
                                        parallel_iterations=None,
                                        swap_memory=False,
                                        time_major=False,
                                        scope=None
                                    )
        return fw_outs, bw_outs, fw_final, bw_final
    
    def create_rnn_cell(self, rnn_size):
        cell = tf.contrib.rnn.BasicLSTMCell(rnn_size, state_is_tuple=True, forget_bias=0.0, reuse=False)
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.drop_out_prob_keep)
        return cell

In [6]:
from tensorflow.losses import Reduction
from model import adict

W0904 09:32:13.288387 139669057967936 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W0904 09:32:13.319870 139669057967936 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF_GPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0904 09:32:13.320659 139669057967936 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF_GPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 09:32:13.322491 139669057967936 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF_GPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0904 09:32:14.102209 1396

In [94]:
def InferSentiTrainGraph(senti_model,batch_size, max_word_num, embedding_dim, hidden_dim, sent_num):
    sent_x = tf.placeholder(tf.float32, shape = [batch_size, max_word_num, embedding_dim])
    sent_y = tf.placeholder(tf.float32, shape = [batch_size, sent_num])
#     x_len = tf.placeholder(tf.int32, shape=[batch_size])
    
#     fw_init = senti_model.fw_cell.zero_state(batch_size, dtype=tf.float32)
#     bw_init = senti_model.bw_cell.zero_state(batch_size, dtype=tf.float32)
#     fw_outs, bw_outs, fw_final, bw_final = senti_model(sent_x, x_len, fw_init, bw_init)
#     print("fw_out:", fw_outs)
#     fw_final = fw_final[-1]
#     bw_final = bw_final[-1]
    
#     # add to final tensor
#     out_merge = fw_final + bw_final
    
    # concat two tensor
#     senti_features = tf.concat([fw_final+tf.reduce_max(fw_outs, axis=1),                            
#                                 bw_final+tf.reduce_max(fw_outs, axis=1)], axis = -1)
    
#     # max_pooling two tensor
#     out_merge = tf.reduce_max(
#         tf.transpose(
#             tf.identity([fw_final, bw_final]),
#             [1, 0, 2]
#         ),
#         axis = 1
#     )
    senti_features = senti_model(sent_x)
    
    w_t = tf.Variable(tf.random_uniform([1100, sent_num], -1.0, 1.0), name="w_t")
    w_t = tf.Print(w_t, [w_t[0], w_t[1], w_t[2]], message="w_t:")
    b_t = tf.Variable(tf.constant(0.01, shape=[sent_num]), name="b_t")
    b_t = tf.Print(b_t, [b_t[0], b_t[1]], message="b_t:")
    senti_rst = tf.nn.relu(
        tf.nn.xw_plus_b(senti_features, w_t, b_t, name="senti_rst")
    )
    
#     classifier = tf.layers.Dense(sent_num, activation=tf.nn.relu, trainable=True)
#     senti_rst = classifier(senti_features)
    
    sent_scores = tf.nn.softmax(senti_rst, axis=1)
    sent_pred = tf.argmax(sent_scores, 1, name="predictions")
    sent_loss = tf.losses.softmax_cross_entropy(
                        sent_y,
                        senti_rst,
                        weights=1.0,
                        label_smoothing=0,
                        scope=None,
                        loss_collection=tf.GraphKeys.LOSSES,
                        reduction=Reduction.SUM_BY_NONZERO_WEIGHTS
                    )
    sent_correct_predictions = tf.equal(sent_pred, tf.argmax(sent_y, 1))
    sent_acc = tf.reduce_mean(tf.cast(sent_correct_predictions, "float"), name="accuracy")
    sent_global_step = tf.Variable(0, name="global_step", trainable=False)
    sent_train_op = tf.train.AdagradOptimizer(0.5).minimize(sent_loss, sent_global_step)        
    return adict(
                sent_x = sent_x,
                sent_y = sent_y,
                feature = senti_features,
                sent_scores = sent_scores,
                sent_pred = sent_pred,
                sent_loss = sent_loss,
                sent_acc = sent_acc,
                sent_global_step = sent_global_step,
                sent_train_op = sent_train_op
            )


In [95]:
def texts2matrix(texts, c2vec, max_sent_len, embedding_dim):
    def words2arr(words):
        rst = np.zeros([max_sent_len, embedding_dim])
        arr = c2vec.vectorize_words(words)
        rst[:len(arr)] = arr
        return rst
    embedding = np.array([words2arr(words).tolist() for words in texts])
    return embedding

In [96]:
def TrainSentiModel(sess, saver, summary_writer, logger, train_model, c2vec, 
                    train_reader, valid_reader, test_reader,
                    train_batch, test_batch, embedding_dim):
    train_epoch = 50
    for epoch in range(train_epoch):
        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        count = 0
        for texts, labels, ratings, lens  in train_reader.iter():
            data_X = texts2matrix(texts, c2vec, train_reader.max_sent_len, embedding_dim)
            feed_dic = {
                        train_model.sent_x: data_X, 
                        train_model.sent_y: labels                     
            }
            _, step, loss, acc, pred, score, feat= sess.run(
                                        [train_model.sent_train_op, 
                                         train_model.sent_global_step, 
                                         train_model.sent_loss, 
                                         train_model.sent_acc,
                                         train_model.sent_pred, 
                                         train_model.sent_scores,
                                         train_model.feature
                                        ], 
                                        feed_dic)
            print("prediction:", pred)
            print("scores:", score[:3].tolist())
            logger.info("feat1:" + str(feat[0].tolist()[:10]) + "\nfeat2:" + str(feat[1].tolist()[:10]) + "\nfeat3:" + str(feat[2].tolist()[:10]))
            count += 1
            sum_loss += loss
            sum_acc += acc
            if count % 10 == 9:
                sum_loss = sum_loss / 10
                sum_acc = sum_acc / 10
                ret_acc = sum_acc
                print('%6d: %d [%5d/%5d], train_loss/accuracy = %6.8f/%6.7f' % (step,
                                                        epoch, count,
                                                        train_reader.label.shape[0],
                                                        sum_loss, sum_acc,
                                                        ))
                logger.info('%6d: %d [%5d/%5d], train_loss/accuracy = %6.8f/%6.7f' % (step,
                                                        epoch, count,
                                                        train_reader.label.shape[0],
                                                        sum_loss, sum_acc,
                                                        ))
                sum_acc = 0.0
                sum_loss = 0.0
                summary = tf.Summary(value=[
                    tf.Summary.Value(tag="step_train_loss", simple_value=sum_loss),
                    tf.Summary.Value(tag="step_train_accuracy", simple_value=sum_acc),
                ])
                summary_writer.add_summary(summary, step)

        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        for texts, labels, ratings, lens  in valid_reader.iter():
            data_X = texts2matrix(texts, c2vec, valid_reader.max_sent_len, embedding_dim)
            feed_dic = {
                        train_model.sent_x: data_X, 
                        train_model.sent_y: labels                      
            }
            _, step, loss, acc = sess.run(
                                        [
                                         train_model.sent_loss, 
                                         train_model.sent_acc], 
                                        feed_dic)
            sum_loss += loss
            sum_acc += acc
        sum_loss = sum_loss / (1.0*valid_reader.label.shape[0])
        sum_acc = sum_acc / (1.0*valid_reader.label.shape[0])
        ret_acc = sum_acc
        print('[%5d/%5d], valid_loss/accuracy = %6.8f/%6.7f' % (epoch, train_epoch,
                                                                sum_loss, sum_acc,
                                                                ))
        logger.info('[%5d/%5d], valid_loss/accuracy = %6.8f/%6.7f' % (epoch, train_epoch,
                                                                sum_loss, sum_acc,
                                                                ))
        summary = tf.Summary(value=[
                    tf.Summary.Value(tag="epoch_valid_loss", simple_value=sum_loss),
                    tf.Summary.Value(tag="epoch_valid_accuracy", simple_value=sum_acc),
                ])
        summary_writer.add_summary(summary, epoch)
        sum_acc = 0.0
        sum_loss = 0.0
        saver.save(sess, "IMDBReports/")
    
    # for validation
    sum_acc = 0.0
    sum_loss = 0.0
    for texts, labels, ratings, lens  in valid_reader.iter():
        data_X = texts2matrix(texts, c2vec, valid_reader.max_sent_len, embedding_dim)
        feed_dic = {
                    train_model.sent_x: data_X, 
                    train_model.sent_y: labels                       
        }
        _, step, loss, acc = sess.run(
                                    [
                                     train_model.sent_loss, 
                                     train_model.sent_acc], 
                                    feed_dic)
        sum_loss += loss
        sum_acc += acc
    print('Training Ended! #test_loss/accuracy = %6.8f/%6.7f' % (sum_loss, sum_acc))
    logger.info('Training Ended! #test_loss/accuracy = %6.8f/%6.7f' % (sum_loss, sum_acc))

In [97]:
from logger import MyLogger
logger = MyLogger("IMDBSentiModel")

In [98]:
gpu_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.4
# device = '/GPU:0'
with tf.Graph().as_default() as g:
    with tf.Session(graph=g, config=gpu_config) as sess:
        with tf.device('/GPU:0'):
            c2vec = chars2vec.load_model('eng_300')
#             senti_model = SentimentModel( FLAGS.hidden_dim )
            senti_model = SentiModel(max_word_length = max([train_reader.max_sent_len, 
                                                             valid_reader.max_sent_len, 
                                                             test_reader.max_sent_len]),
                                       kernels = eval(FLAGS.kernels), 
                                       kernel_features = eval(FLAGS.kernel_features),
                                       num_highway_layers = FLAGS.highway_layers,
                                       embedding_dim = FLAGS.embedding_dim
                                      )
            IMDB_train_graph =  InferSentiTrainGraph(
                                                         senti_model = senti_model,
                                                         batch_size = FLAGS.batch_size, 
                                                         max_word_num = max([train_reader.max_sent_len, 
                                                                             valid_reader.max_sent_len, 
                                                                             test_reader.max_sent_len]), 
                                                         embedding_dim = FLAGS.embedding_dim, 
                                                         hidden_dim = FLAGS.hidden_dim, 
                                                         sent_num = FLAGS.sent_num
                                                    )
            val_list1 = tf.global_variables()
            saver = tf.train.Saver(val_list1, max_to_keep=4)
            sess.run(tf.variables_initializer(val_list1))
            summary_writer = tf.summary.FileWriter("IMDBReports/", graph=sess.graph)
            TrainSentiModel(sess, saver, summary_writer, logger, IMDB_train_graph, c2vec, 
                    train_reader, valid_reader, test_reader,
                    FLAGS.batch_size, FLAGS.batch_size, FLAGS.embedding_dim)
        

input_cnn: Tensor("Embedding/CNN_OUT/add_7:0", shape=(20, 1100), dtype=float32, device=/device:GPU:0)


I0904 10:59:24.029239 139669057967936 logger.py:24] feat1:[-0.05206857994198799, -0.08402467519044876, 0.15650062263011932, -0.06343629956245422, 0.043652597814798355, 0.15307141840457916, 0.11379000544548035, 0.1216433197259903, -0.03424294665455818, 0.13517090678215027]
feat2:[-0.05564611777663231, -0.08265110850334167, 0.15593001246452332, -0.06655476242303848, 0.036800045520067215, 0.15139809250831604, 0.11387063562870026, 0.1212642565369606, -0.0357990488409996, 0.1353151500225067]
feat3:[-0.05637918785214424, -0.08173474669456482, 0.1536109447479248, -0.06696655601263046, 0.0380835197865963, 0.15130707621574402, 0.11331826448440552, 0.12115437537431717, -0.03607778623700142, 0.1364312320947647]


prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]


I0904 10:59:25.331845 139669057967936 logger.py:24] feat1:[-0.05495750159025192, -0.08257828652858734, 0.1539868712425232, -0.06732626259326935, 0.03559203818440437, 0.14916929602622986, 0.11045101284980774, 0.11635219305753708, -0.0359332337975502, 0.13266493380069733]
feat2:[-0.05497051775455475, -0.07953786104917526, 0.15428586304187775, -0.06634335219860077, 0.03758054971694946, 0.15624096989631653, 0.11325681954622269, 0.12448704987764359, -0.03404149413108826, 0.13678698241710663]
feat3:[-0.05494477227330208, -0.08462868630886078, 0.1529572308063507, -0.06628786772489548, 0.03525780141353607, 0.14889554679393768, 0.11219670623540878, 0.12163718044757843, -0.03589022904634476, 0.13410022854804993]


prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]


...

I0904 10:59:58.317412 139669057967936 logger.py:24] feat1:[-0.05497090518474579, -0.08188585191965103, 0.1557251363992691, -0.06287387013435364, 0.043568048626184464, 0.14916695654392242, 0.11360005289316177, 0.11871036142110825, -0.03764397278428078, 0.13871626555919647]
feat2:[-0.05494819954037666, -0.08239865303039551, 0.15486904978752136, -0.06656527519226074, 0.03485582023859024, 0.15301136672496796, 0.11401316523551941, 0.11877457052469254, -0.034981194883584976, 0.13684138655662537]
feat3:[-0.05771902948617935, -0.08080293983221054, 0.15476781129837036, -0.06896647065877914, 0.03516383469104767, 0.15418003499507904, 0.11176930367946625, 0.12178260833024979, -0.03074355237185955, 0.13571037352085114]


prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]


I0904 10:59:59.429717 139669057967936 logger.py:24] feat1:[-0.058496106415987015, -0.0838417336344719, 0.15585820376873016, -0.06895992159843445, 0.03682806342840195, 0.15054836869239807, 0.11164051294326782, 0.11784780770540237, -0.03529087454080582, 0.13771158456802368]
feat2:[-0.05499276891350746, -0.07946449518203735, 0.15631233155727386, -0.06805378943681717, 0.038314223289489746, 0.15330997109413147, 0.11318516731262207, 0.12267043441534042, -0.0345279723405838, 0.13785259425640106]
feat3:[-0.05227978155016899, -0.08245579898357391, 0.1545206606388092, -0.06435641646385193, 0.0391136109828949, 0.14765411615371704, 0.1123533844947815, 0.11599244922399521, -0.036255039274692535, 0.13678058981895447]


prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]


KeyboardInterrupt: 